Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
import matplotlib.pyplot as plt
from IPython import display

%matplotlib notebook

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'Dataset/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [4]:
train_labels.shape

(200000,)

Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [14]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [15]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [ ]:
graphn=tf.Graph()
with graphn.as_default():
    X=tf.placeholder(tf.float32,shape=(None,image_size*image_size))
    y_=tf.placeholder(tf.float32,shape=(None,10))
    hidden_weights=tf.Variable(tf.truncated_normal(shape=(image_size*image_size,1024)))
    hidden_bias=tf.Variable(tf.zeros([1024]))
    z1=tf.matmul(X,hidden_weights)+hidden_bias
    y1=tf.nn.relu(z1)
    output_weights=tf.Variable(tf.truncated_normal(shape=(1024,num_labels)))
    output_bias=tf.Variable(tf.zeros([num_labels]))
    z2=tf.matmul(y1,output_weights)+output_bias
    loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=z2))+0.001*(tf.nn.l2_loss(output_weights)+tf.nn.l2_loss(hidden_weights))
    optimizer=tf.train.GradientDescentOptimizer(0.1).minimize(loss)
    predict=tf.nn.softmax(z2)

In [ ]:
batch_size=128

In [ ]:
num_steps=50000
with tf.Session(graph=graphn) as sess:
    #Initialise the global variables
    tf.global_variables_initializer().run()
    print('Started Training')
    for i in range(0,num_steps):
        offset = (i * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        sess.run(optimizer,feed_dict={X:batch_data,y_:batch_labels})
         
        if(i%5000==0):
            print('Loss: '+str(sess.run(loss,feed_dict={X:batch_data,y_:batch_labels})))
    print(accuracy(sess.run(predict,feed_dict={X:train_dataset,y_:train_labels}),train_labels))
    print(accuracy(sess.run(predict,feed_dict={X:test_dataset,y_:test_labels}),test_labels))
    print(accuracy(sess.run(predict,feed_dict={X:valid_dataset,y_:valid_labels}),valid_labels))


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [16]:
graphn=tf.Graph()
with graphn.as_default():We also make use of max pooling layers of size 2x2 to help us arrive at a scale invariant representation of the image. Hence, we can detect objects in an image regardless of where they are located.

    
    X=tf.placeholder(tf.float32,shape=(None,image_size*image_size))
    y_=tf.placeholder(tf.float32,shape=(None,10))
    hidden_weights=tf.Variable(tf.truncated_normal(shape=(image_size*image_size,1024)))
    hidden_bias=tf.Variable(tf.zeros([1024]))
    z1=tf.matmul(X,hidden_weights)+hidden_bias
    y1=tf.nn.relu(z1)
    y1=tf.nn.dropout(y1,keep_prob=0.9)
    output_weights=tf.Variable(tf.truncated_normal(shape=(1024,num_labels)))
    output_bias=tf.Variable(tf.zeros([num_labels]))
    z2=tf.matmul(y1,output_weights)+output_bias
    loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=z2))
    optimizer=tf.train.GradientDescentOptimizer(0.1).minimize(loss)
    #Our predict is a bit different. It does not implement dropout
    y1_predict=tf.nn.relu(z1)
    z2_predict=tf.matmul(y1_predict,output_weights)+output_bias
    predict=tf.nn.softmax(z2)

In [142]:
batch_size=1024


In [53]:
num_steps=10000
with tf.Session(graph=graphn) as sess:
    #Initialise the global variables
    tf.global_variables_initializer().run()
    print('Started Training')
    for i in range(0,num_steps):
        offset = (i * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        sess.run(optimizer,feed_dict={X:batch_data,y_:batch_labels})  
        if(i%500==0):
            print('Loss: '+str(sess.run(loss,feed_dict={X:batch_data,y_:batch_labels})))
    print(accuracy(sess.run(predict,feed_dict={X:train_dataset,y_:train_labels}),train_labels))
    print(accuracy(sess.run(predict,feed_dict={X:test_dataset,y_:test_labels}),test_labels))
    print(accuracy(sess.run(predict,feed_dict={X:valid_dataset,y_:valid_labels}),valid_labels))

Started Training


TypeError: Cannot interpret feed_dict key as Tensor: Tensor Tensor("Placeholder:0", shape=(?, 784), dtype=float32) is not an element of this graph.

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [156]:
graphd=tf.Graph()
with graphd.as_default():
    X=tf.placeholder(tf.float32,shape=(None,image_size*image_size))
    y_=tf.placeholder(tf.float32,shape=(None,10))
    p=tf.placeholder(tf.float32)
    # Hidden Layer 1
    hidden1_weights=tf.Variable(tf.truncated_normal(shape=(image_size*image_size,4096)))
    hidden1_bias=tf.Variable(tf.zeros([4096]))
    z1=tf.matmul(X,hidden1_weights)+hidden1_bias
    y1=tf.nn.relu(z1)
    y1=tf.nn.dropout(y1,keep_prob=p)
    #Hidden Layer 2
    hidden2_weights=tf.Variable(tf.truncated_normal(shape=(4096,2048)))
    hidden2_bias=tf.Variable(tf.zeros([2048]))
    z2=tf.matmul(y1,hidden2_weights)+hidden2_bias
    y2=tf.nn.relu(z2)
    y2=tf.nn.dropout(y2,keep_prob=p)
    #Hidden Layer 3
    hidden3_weights=tf.Variable(tf.truncated_normal(shape=(2048,1024)))
    hidden3_bias=tf.Variable(tf.zeros([1024]))
    z3=tf.matmul(y2,hidden3_weights)+hidden3_bias
    y3=tf.nn.relu(z3)
    y3=tf.nn.dropout(y3,keep_prob=p)
    #Output Layer  
    output_weights=tf.Variable(tf.truncated_normal(shape=(1024,num_labels)))
    output_bias=tf.Variable(tf.zeros([num_labels]))
    out=tf.matmul(y3,output_weights)+output_bias
    regularizer=0.0001*(tf.nn.l2_loss(hidden1_weights)+tf.nn.l2_loss(hidden2_weights)+tf.nn.l2_loss(hidden3_weights)+tf.nn.l2_loss(output_weights))
    loss=tf.reduce_mean(tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=out))+regularizer)
    
    global_step = tf.Variable(0)  # count the number of steps taken.
    start_learning_rate = 0.5
    learning_rate = tf.train.exponential_decay(start_learning_rate, global_step, 100000, 0.96, staircase=True)
    optimizer=tf.train.AdamOptimizer().minimize(loss,global_step=global_step)
    
    predict=tf.nn.softmax(out)

In [157]:
num_steps=150000
with tf.Session(graph=graphd) as sess:
    #Initialise the global variables
    tf.global_variables_initializer().run()
    print('Started Training')
    for i in range(0,num_steps):
        offset = (i * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        sess.run(optimizer,feed_dict={X:batch_data,y_:batch_labels,p:0.5})  
        if(i%1000==0):
            print('Loss: '+str(sess.run(loss,feed_dict={X:batch_data,y_:batch_labels,p:1})))
    print(accuracy(sess.run(predict,feed_dict={X:train_dataset,y_:train_labels,p:1}),train_labels))
    print(accuracy(sess.run(predict,feed_dict={X:test_dataset,y_:test_labels,p:1}),test_labels))
    print(accuracy(sess.run(predict,feed_dict={X:valid_dataset,y_:valid_labels,p:1}),valid_labels))

Started Training
Loss: 198445.0
Loss: 1367.22
Loss: 532.056
Loss: 529.527
Loss: 529.457
Loss: 529.209
Loss: 528.79
Loss: 528.106
Loss: 526.987
Loss: 525.163
Loss: 522.216
Loss: 517.502
Loss: 510.072
Loss: 498.644
Loss: 481.745
Loss: 458.135
Loss: 427.622
Loss: 390.635
Loss: 350.282
Loss: 313.546
Loss: 283.939
Loss: 263.003
Loss: 248.581
Loss: 237.495
Loss: 227.01
Loss: 215.633
Loss: 203.159
Loss: 189.794
Loss: 175.69
Loss: 161.803
Loss: 148.554
Loss: 135.261
Loss: 122.808
Loss: 111.71
Loss: 101.604
Loss: 91.9208
Loss: 83.3241
Loss: 74.956
Loss: 66.595
Loss: 58.3765
Loss: 50.385
Loss: 42.6675
Loss: 35.2866
Loss: 28.2337
Loss: 21.779
Loss: 16.3655
Loss: 11.8425
Loss: 8.29286
Loss: 5.84609
Loss: 4.25894
Loss: 3.31715
Loss: 1.51481
Loss: 0.70172
Loss: 0.565169
Loss: 0.499792
Loss: 0.436978
Loss: 0.463918
Loss: 0.386759
Loss: 0.471461
Loss: 0.338141
Loss: 0.361556
Loss: 0.359772
Loss: 0.375632
Loss: 0.323349
Loss: 0.343734
Loss: 0.328096
Loss: 0.354403
Loss: 0.345307
Loss: 0.341259
Loss: 0.

ResourceExhaustedError: OOM when allocating tensor with shape[200000,4096]
	 [[Node: dropout/random_uniform/RandomUniform = RandomUniform[T=DT_INT32, dtype=DT_FLOAT, seed=0, seed2=0, _device="/job:localhost/replica:0/task:0/gpu:0"](dropout/Shape)]]
	 [[Node: Softmax/_47 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_63_Softmax", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'dropout/random_uniform/RandomUniform', defined at:
  File "/home/metal_geek/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/metal_geek/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/metal_geek/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/metal_geek/

In [160]:
sess=tf.Session(graph=graphd)
print(accuracy(sess.run(predict,feed_dict={X:train_dataset,y_:train_labels,p:1}),train_labels))
print(accuracy(sess.run(predict,feed_dict={X:test_dataset,y_:test_labels,p:1}),test_labels))
print(accuracy(sess.run(predict,feed_dict={X:valid_dataset,y_:valid_labels,p:1}),valid_labels))

FailedPreconditionError: Attempting to use uninitialized value Variable
	 [[Node: Variable/read = Identity[T=DT_FLOAT, _class=["loc:@Variable"], _device="/job:localhost/replica:0/task:0/gpu:0"](Variable)]]
	 [[Node: Softmax/_5 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_54_Softmax", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'Variable/read', defined at:
  File "/home/metal_geek/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/metal_geek/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/metal_geek/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/metal_geek/anaconda3/lib/python3.6/site-packages/traitlets/config/application.